# 01A Data Cleansing Weather

## Imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
%matplotlib inline

## Read Data

In [2]:
weather_df = pd.read_csv('../west_nile/west_nile/input/weather.csv')

## One Hot Encode CodeSum Data

In [3]:
codes = {code: [0]*weather_df.shape[0] for codes in weather_df.CodeSum.unique() for code in codes.split()}

weather_df = weather_df.join(pd.DataFrame(codes))
for i, codesum in enumerate(weather_df.CodeSum):
    for code in codesum.split():
        weather_df.at[i, code] = 1
        
codes_df = weather_df.groupby('Date', as_index=True)[list(codes.keys())].max()

## Calculate Missing Tavg Values

In [4]:
missing_Tavg = weather_df[weather_df.Tavg == 'M'].index

weather_df.loc[missing_Tavg, 'Tavg'] = round((weather_df.loc[missing_Tavg, 'Tmax'] 
                                              + weather_df.loc[missing_Tavg, 'Tmin'])/2)

## Convert Tavg to Int

In [5]:
weather_df.Tavg = weather_df.Tavg.astype(int)

In [6]:
temperature_df = weather_df.groupby('Date')['Tavg','Tmax','Tmin'].mean()

## Drop Unnecessary Columns
- Incorporate Adam's Work

In [7]:
cols_drop = ['Depart','DewPoint','WetBulb','Heat','Cool','Sunrise','Sunset','CodeSum','Depth','Water1',
             'SnowFall','PrecipTotal','StnPressure','SeaLevel','ResultSpeed','ResultDir','AvgSpeed']
weather_df = weather_df.drop(labels=cols_drop, axis=1)

## Export codes_df and temperature_df

In [8]:
with open('../Cleansed_Data/codes_df.pkl', 'wb') as f:
    pickle.dump(codes_df, f)
    
with open('../Cleansed_Data/temperature_df.pkl', 'wb') as f:
    pickle.dump(temperature_df, f)